In [ ]:
#Import packages

import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (roc_curve, roc_auc_score, auc, log_loss,
                             precision_score, recall_score, f1_score,
                             accuracy_score, classification_report,
                             ConfusionMatrixDisplay, confusion_matrix)

In [ ]:
# If using Google Colab.

from google.colab import drive
drive.mount('/content/drive')

##### The ZEMO python package [https://pypi.org/project/ZEMO/] can be used to compute Zernike moments for your images. This library is described in a research paper you can find online [[arxiv:2308.13562](https://arxiv.org/abs/2308.13562v2)].

In [ ]:
zm_nongalaxy = pd.read_csv('/path/to/your/direcotry/zms_data.csv')
zm_galaxy = pd.read_csv('/path/to/your/direcotry/zms_data.csv')

zm_nongalaxy.drop("Unnamed: 0", axis = 1, inplace = True)
zm_galaxy.drop("Unnamed: 0", axis = 1, inplace = True)

zms1 = np.array(zm_nongalaxy)
zmg1 = np.array(zm_galaxy)

all_zm_data = np.concatenate([zmg1,zms1])
len(zmg1), len(zms1), len(all_zm_data)

In [ ]:
# labels due to the length of data 

galaxies_labels = np.zeros(780)
nongalaxy_labels = np.ones(545)
all_labels = np.concatenate([galaxies_labels, nongalaxy_labels])
len(all_labels)

In [ ]:
# Check the class weights 

total_samples = len(all_zm_data)

class_weights = {
    0: total_samples / (3*len(zmg1)),  # Class 0 weight
    1: total_samples / (3*len(zms1)),   # Class 1 weight
}

print("Class Weights based on Inverse Class Frequency:")
print("Class 1 Weight:", class_weights[0])
print("Class 2 Weight:", class_weights[1])

In [ ]:
# Run the model for 10 iterations and calculate the standard deviation.
# You can save the performance materials and the models outputs. 

y_test_list=[]
models=[]
Y_pred=[]
accs=[]
cons=[]
test_indx = []

for i in range (10):
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(all_zm_data, all_labels, np.arange(len(all_labels)), test_size=0.25, shuffle=True, random_state=None)
    y_test_list.append(y_test)
    test_indx.append(test_indices)

    class_weights = {0: len(all_zm_data) / (2*len(zmg1)), 1: len(all_zm_data) / (2*len(zms1))}

    model = SVC(kernel='rbf', probability=True, C=1.5, gamma='scale',class_weight=class_weights)
    gz2_training_model = model.fit(X_train, y_train)
    models.append(gz2_training_model)

    y_pred = model.predict(X_test)
    Y_pred.append(y_pred)

    con0 = metrics.confusion_matrix(y_test, y_pred)
    cons.append(con0)

    acc = metrics.accuracy_score(y_test, y_pred)
    accs.append(acc)

In [ ]:
# To save any output you need.

output_el_path = '/path/to/your/direcotry'
import os
pickle_el_filename = 'performances_of_galaxy_nonegalaxy.pickle'
pickle_el_filepath = os.path.join(output_el_path, pickle_el_filename)

with open(pickle_el_filepath, 'wb') as pickle_file:
    pickle.dump(accs, pickle_file)


In [ ]:
print(np.mean(accs), np.std(accs))
plt.plot(accs)